In [ ]:
# 필요한 라이브러리 import
import time
import os
from Raspbot_Lib import Raspbot
from ipywidgets import interact
import ipywidgets as widgets
import sys

sys.path.append("/home/pi/software/oled_yahboom/")
from yahboom_oled import *

In [ ]:
# Raspbot 및 OLED 객체 생성
bot = Raspbot()
oled = Yahboom_OLED(debug=False)

In [ ]:
# ============================================
# 초음파 거리 측정 관련 함수들
# ============================================


def init_ultrasonic_sensor(bot_instance):
    """
    초음파 센서를 초기화하고 활성화합니다.

    Args:
        bot_instance: Raspbot 객체

    Returns:
        bool: 초기화 성공 여부
    """
    if bot_instance is None:
        print("Error: Raspbot instance is not available.")
        return False

    bot_instance.Ctrl_Ulatist_Switch(1)
    time.sleep(0.1)  # 센서 초기화 대기 시간
    return True


def read_ultrasonic_distance(bot_instance):
    """
    초음파 센서로부터 거리 값을 읽어옵니다.

    Args:
        bot_instance: Raspbot 객체

    Returns:
        int: 거리 값 (mm 단위), 실패 시 -1
    """
    if bot_instance is None:
        return -1

    try:
        # 상위 바이트와 하위 바이트를 읽어옴
        diss_H = bot_instance.read_data_array(0x1B, 1)[0]
        diss_L = bot_instance.read_data_array(0x1A, 1)[0]
        # 16비트 값으로 조합 (상위 바이트 << 8 | 하위 바이트)
        distance = (diss_H << 8) | diss_L
        return distance
    except Exception as e:
        print(f"Error reading distance: {e}")
        return -1


def display_distance_on_oled(oled_instance, distance):
    """
    OLED 화면에 거리 정보를 표시합니다.

    Args:
        oled_instance: Yahboom_OLED 객체
        distance: 표시할 거리 값 (mm)
    """
    if oled_instance is None:
        return

    oled_instance.clear()
    distance_str = f"{distance:>10} mm"
    oled_instance.add_line("Ultrasonic Distance:", 1)
    oled_instance.add_line(distance_str, 3)
    oled_instance.refresh()


def cleanup_ultrasonic_sensor(bot_instance):
    """
    초음파 센서를 비활성화하고 정리합니다.

    Args:
        bot_instance: Raspbot 객체
    """
    if bot_instance is None:
        return

    bot_instance.Ctrl_Ulatist_Switch(0)
    # OLED 기본 프로세스 복원
    os.system("python3 /home/pi/software/oled_yahboom/yahboom_oled.py &")
    print("Ultrasonic sensor turned off.")


def main_ultrasonic_measurement(bot_instance, oled_instance, measurement_interval=0.05):
    """
    초음파 거리 측정 메인 루프

    Args:
        bot_instance: Raspbot 객체
        oled_instance: Yahboom_OLED 객체
        measurement_interval: 측정 간격 (초 단위, 기본값: 0.05초)
    """
    # 1. 초음파 센서 초기화
    if not init_ultrasonic_sensor(bot_instance):
        print("Failed to initialize ultrasonic sensor.")
        return

    # 2. OLED 초기화
    oled_instance.init_oled_process()

    try:
        # 3. 메인 측정 루프
        while True:
            # 3-1. 거리 읽기
            distance = read_ultrasonic_distance(bot_instance)

            # 3-2. 거리 값 출력
            if distance >= 0:
                print(f"Ultrasonic Distance: {distance} mm")
                # 3-3. OLED에 거리 표시
                display_distance_on_oled(oled_instance, distance)
            else:
                print("Failed to read distance.")

            # 3-4. 측정 간격 대기
            time.sleep(measurement_interval)

    except KeyboardInterrupt:
        # 4. 사용자 중단 시 정리 작업
        print("\nInterrupted by user.")
    finally:
        # 5. 센서 종료 및 정리
        cleanup_ultrasonic_sensor(bot_instance)


# ============================================
# 메인 실행 함수
# ============================================
def main():
    """메인 실행 함수"""
    main_ultrasonic_measurement(bot, oled, measurement_interval=0.05)


if __name__ == "__main__":
    main()

No matching processes found.
Ultrasonic Distance: 731 mm
Ultrasonic Distance: 867 mm
Ultrasonic Distance: 697 mm
Ultrasonic Distance: 612 mm
Ultrasonic Distance: 697 mm
Ultrasonic Distance: 697 mm
Ultrasonic Distance: 697 mm
Ultrasonic Distance: 748 mm
Ultrasonic Distance: 731 mm
Ultrasonic Distance: 323 mm
Ultrasonic Distance: 595 mm
Ultrasonic Distance: 680 mm
Ultrasonic Distance: 680 mm
Ultrasonic Distance: 697 mm
Ultrasonic Distance: 408 mm
Ultrasonic Distance: 323 mm
Ultrasonic Distance: 221 mm
Ultrasonic Distance: 187 mm
Ultrasonic Distance: 204 mm
Ultrasonic Distance: 170 mm
Ultrasonic Distance: 187 mm
Ultrasonic Distance: 221 mm
Ultrasonic Distance: 221 mm
Ultrasonic Distance: 221 mm
Ultrasonic Distance: 221 mm
Ultrasonic Distance: 238 mm
Ultrasonic Distance: 204 mm
Ultrasonic Distance: 238 mm
Ultrasonic Distance: 238 mm
Ultrasonic Distance: 221 mm
Ultrasonic Distance: 204 mm
Ultrasonic Distance: 221 mm
Ultrasonic Distance: 204 mm
Ultrasonic Distance: 187 mm
Ultrasonic Distance

In [ ]:
# 프로그램 종료 시 정리 작업
# 1. 초음파 센서 비활성화
cleanup_ultrasonic_sensor(bot)

# 2. 객체 삭제 (다른 프로그램에서 Raspbot 라이브러리 사용 시 충돌 방지)
del bot